## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,7/6/16,Citibike to Work,1.20,NaN,10.0,0.17,7.20,Citibike
1,11/11/16,Citibike from Work,1.00,NaN,10.0,0.17,6.00,Citibike
2,11/11/16,Citibike to Work,1.30,NaN,11.0,0.18,7.09,Citibike
3,8/24/16,Citibike from Work,0.83,NaN,8.0,0.13,6.23,Citibike
4,9/15/19,Roosevelt Island,6.24,1.0,6.0,1.10,5.67,Specialized
...,...,...,...,...,...,...,...,...
348,4/12/20,CP BL x 2,13.81,1.0,48.0,1.80,7.67,Specialized
349,10/25/16,Citibike to Work,1.46,NaN,10.0,0.17,8.76,Citibike
350,10/1/19,CP BL,7.43,1.0,6.0,1.10,6.75,Specialized
351,9/20/16,Citibike from Work Midday,0.90,NaN,12.0,0.20,4.50,Citibike


In [8]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [9]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,1.20,0.17,7.20,Citibike
1,1.00,0.17,6.00,Citibike
2,1.30,0.18,7.09,Citibike
3,0.83,0.13,6.23,Citibike
4,6.24,1.10,5.67,Specialized
...,...,...,...,...
348,13.81,1.80,7.67,Specialized
349,1.46,0.17,8.76,Citibike
350,7.43,1.10,6.75,Specialized
351,0.90,0.20,4.50,Citibike


In [7]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
#new2_df = all_df[['Miles', 'Duration', 'Speed', 'Type']]
#new2_df

,Miles,Duration,Speed
0,1.20,0.17,7.20
1,1.00,0.17,6.00
2,1.30,0.18,7.09
3,0.83,0.13,6.23
4,6.24,1.10,5.67
...,...,...,...
348,13.81,1.80,7.67
349,1.46,0.17,8.76
350,7.43,1.10,6.75
351,0.90,0.20,4.50


In [10]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [13]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [14]:
# Checking training data shape
X_train.shape

(264, 3)

In [15]:
# Checking testing data shape
X_test.shape

(89, 3)

In [16]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [17]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [20]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [21]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

1.0
